In [1]:
import sys
import os
import glob
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv('../processed_data/train_v3.csv')
test = pd.read_csv('../processed_data/test_v3.csv')

In [4]:
train.columns

Index(['id', 'y', 'location', 'access', 'layout', 'age', 'direction', 'area',
       'floor', 'bath_toilet', 'kitchen', 'broadcast_com', 'facilities',
       'parking', 'enviroment', 'structure', 'contact_period', 'log_y',
       'max_floor', 'room_num', 'L', 'D', 'K', 'S', 'north', 'walk_time',
       'min_time', 'avg_time', '23ku', '23ku_mean_std'],
      dtype='object')

In [6]:
pd.set_option('display.max_columns', 100)
train.head()

,id,y,location,access,layout,age,direction,area,floor,bath_toilet,kitchen,broadcast_com,facilities,parking,enviroment,structure,contact_period,log_y,max_floor,room_num,L,D,K,S,north,walk_time,min_time,avg_time,23ku,23ku_mean_std
0,1,75000,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,117,南東,20.01,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,7,2年間,11.225243,12,1,0,0,1,0,0,"[4, 14, 7]",4,8.333333,北区,-0.686553
1,2,76000,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,538,NaN,16.50,5,専用トイレ／\tシャワー／\t温水洗浄便座,ガスコンロ／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 1283m,2,2年間,11.238489,10,1,0,0,0,0,0,"[5, 9, 20]",5,11.333333,中央区,1.684829
2,3,110000,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,1K,102,南,22.05,12,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\t光ファイバー／\tCSアンテナ／\tBSアンテナ,エアコン付\tウォークインクローゼット\tシューズボックス／\tバルコニー／\tフローリング...,"駐輪場\t空有\tバイク置き場\t空有\t駐車場\t近隣\t30,000円(税込)\t距離100m",【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,7,2年間,11.608236,15,1,0,0,1,0,0,"[6, 7, 17]",6,10.000000,渋谷区,1.421175
3,4,150000,東京都杉並区高円寺南２丁目23-2,総武線・中央線（各停）\t高円寺駅\t徒歩9分\t\t丸ノ内線(池袋－荻窪)\t新高円寺駅\...,2LDK,352,南,60.48,3,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座／\t洗面台独立,ガスコンロ／\t給湯,インターネット対応／\t光ファイバー,エアコン付\tシューズボックス／\tバルコニー／\t2面採光／\t室内洗濯機置場／\tエレベ...,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【スーパー】 225m\t【スーパー】 448m\t【スーパー】 619m\t【スーパー】 ...,7,2年間\t※この物件は\t定期借家\tです。,11.918391,4,2,1,1,1,0,0,"[9, 3, 14]",3,8.666667,杉並区,-0.738958
4,5,74000,東京都葛飾区金町３丁目7-2,京成金町線\t京成金町駅\t徒歩5分\t\t常磐線\t金町(東京都)駅\t徒歩7分\t\t京...,2DK,379,南,39.66,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能,給湯／\t独立キッチン,NaN,バルコニー／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水,"駐車場\t近隣\t17,000円(税込)\t距離300m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 193m\t【スーパー】 298m\t【スーパー】 660m\t【スーパー】 ...,1,2年間,11.211820,2,2,0,1,1,0,0,"[5, 7, 17]",5,9.666667,葛飾区,-1.280757


### directionの前処理2

- Count Encodingが有効らしいのでやってみた

In [3]:
def makeCountFull(train, test, categorical_features=None, report=True):
    add_cols = categorical_features
    if report:
        print('add_cols: ', add_cols)
    for add_col in add_cols:
        train[add_col + '_countall'] = train[add_col].map(pd.concat([train[add_col], test[add_col]], ignore_index=True).value_counts(dropna=False))
        test[add_col + '_countall'] = test[add_col].map(pd.concat([train[add_col], test[add_col]], ignore_index=True).value_counts(dropna=False))
    return train, test

In [7]:
cat_features = ['direction']
train, test = makeCountFull(train, test, cat_features)

add_cols:  ['direction']


In [9]:
train['direction_countall'].isnull().sum()

0

In [10]:
test['direction_countall'].isnull().sum()

0

In [14]:
train_v4 = pd.read_csv('../processed_data/train_v4.csv')
test_v4 = pd.read_csv('../processed_data/test_v4.csv')

In [15]:
train_v4.isnull().sum()

id                       0
y                        0
location                 0
access                   0
layout                   0
age                      0
direction             2749
area                     0
floor                    0
bath_toilet            329
kitchen                621
broadcast_com         3440
facilities              82
parking               5287
enviroment            9432
structure                0
contact_period        7120
log_y                    0
max_floor                0
room_num                 0
L                        0
D                        0
K                        0
S                        0
north                    0
walk_time                0
min_time                 0
avg_time                 0
23ku                     0
direction_countall       0
dtype: int64

In [17]:
test_v4.isnull().sum()

id                       0
location                 0
access                   0
layout                   0
age                      0
direction             2808
area                     0
floor                    0
bath_toilet            320
kitchen                635
broadcast_com         3454
facilities              73
parking               5356
enviroment            9241
structure                0
contact_period        6981
max_floor                0
room_num                 0
L                        0
D                        0
K                        0
S                        0
north                    0
walk_time                0
min_time                 0
avg_time                 0
23ku                     0
direction_countall       0
dtype: int64

In [20]:
train_v4[train_v4['direction'].isnull()]

,id,y,location,access,layout,age,direction,area,floor,bath_toilet,kitchen,broadcast_com,facilities,parking,enviroment,structure,contact_period,log_y,max_floor,room_num,L,D,K,S,north,walk_time,min_time,avg_time,23ku,direction_countall
1,2,76000,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,538,NaN,16.50,5,専用トイレ／\tシャワー／\t温水洗浄便座,ガスコンロ／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 1283m,2,2年間,11.238489,10,1,0,0,0,0,0,"[5, 9, 20]",5,11.333333,中央区,5557
7,8,115000,東京都目黒区鷹番１丁目,東急東横線\t学芸大学駅\t徒歩7分\t\t東急東横線\t都立大学駅\t徒歩16分\t\t東...,1K,172,NaN,32.32,1,専用バス／\t専用トイレ／\tバス・トイレ別／\t追焚機能／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン,光ファイバー／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t24時間換気システム\...,NaN,【スーパー】 334m\t【スーパー】 562m\t【スーパー】 694m\t【コンビニ】 ...,7,2年間,11.652687,6,1,0,0,1,0,0,"[7, 16, 20]",7,14.333333,目黒区,5557
11,12,53000,東京都江戸川区北小岩３丁目,京成本線\t京成小岩駅\t徒歩15分\t\t京成本線\t江戸川駅\t徒歩7分,1K,712,NaN,23.10,1,専用トイレ,NaN,NaN,NaN,NaN,NaN,1,NaN,10.878047,2,1,0,0,1,0,0,"[15, 7]",7,11.000000,江戸川区,5557
18,19,310000,東京都新宿区四谷三栄町,総武線・中央線（各停）\t四ツ谷駅\t徒歩5分\t\t丸ノ内線(池袋－荻窪)\t四ツ谷駅\t...,3LDK,72,NaN,70.82,5,専用バス／\t専用トイレ／\tバス・トイレ別／\t追焚機能／\t浴室乾燥機\t／\t温水洗浄...,IHコンロ\t／\tコンロ3口／\tシステムキッチン,光ファイバー／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付／\t床暖房\tウォークインクローゼット\tシューズボックス／\tバルコニー／\t...,駐輪場\t空有,【スーパー】 228m\t【スーパー】 714m\t【コンビニ】 569m\t【コンビニ】 ...,7,2年間,12.644328,11,3,1,1,1,0,0,"[5, 4, 10]",4,6.333333,新宿区,5557
52,53,86500,東京都台東区浅草６丁目,東武スカイツリーライン\t浅草駅\t徒歩12分\t\tつくばエクスプレス\t浅草(ＴＸ)駅\...,1K,64,NaN,26.00,7,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機,コンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tバルコニー／\tフローリング／\t24時間換気システム\t／\t室内洗濯機置場...,駐輪場\t空無\tバイク置き場\t無,NaN,7,NaN,11.367900,10,1,0,0,1,0,0,"[12, 16]",12,14.000000,台東区,5557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31432,31433,75000,東京都世田谷区千歳台,小田急小田原線\t祖師ケ谷大蔵駅\t徒歩16分\t\t京王線\t千歳烏山駅\t/バス(9分)...,1K,271,NaN,23.40,2,バス・トイレ別／\tシャワー／\t温水洗浄便座,給湯,インターネット対応／\tCATV,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場\t都市ガス,駐車場\t無,NaN,4,2年間,11.225243,2,1,0,0,1,0,0,"[16, 5, 13]",5,11.333333,世田谷区,5557
31438,31439,46000,東京都江戸川区江戸川２丁目,都営新宿線\t瑞江駅\t徒歩11分\t\t都営新宿線\t篠崎駅\t徒歩32分\t\t都営新宿...,1R,341,NaN,18.00,2,シャワー,ガスコンロ／\t給湯,NaN,エアコン付\tバルコニー／\tフローリング／\t室外洗濯機置場,"駐車場\t近隣\t13,000円\t距離200m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 255m\t【スーパー】 230m\t【コンビニ】 237m\t【ドラッグスト...,1,2年間,10.736397,2,1,0,0,0,0,0,"[11, 32, 26]",11,23.000000,江戸川区,5557
31442,31443,66000,東京都練馬区富士見台３丁目,西武池袋線\t練馬高野台駅\t徒歩5分\t\t西武池袋線\t富士見台駅\t徒歩14分\t\t...,1R,5,NaN,18.05,1,専用バス／\t専用トイレ／\tバス・トイレ別／\t追焚機能／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン,光ファイバー／\tCATV／\tCSアンテナ／\tBSアンテナ／\tインターネット使用料無料,エアコン付\tシューズボックス／\tフローリング／\t24時間換気システム\t／\t室内洗濯...,駐輪場\t空有\t駐車場\t無,【飲食店】 411m\t【スーパー】 1067m\t【郵便局】 1223m\t【コンビニ】 ...,1,2年間,11.097410,2,1,0,0,0,0,0,"[5, 14, 18]",5,12.333333,練馬区,5557
31452,31453,91500,東京都荒川区町屋５丁目,千代田線\t町屋(東京メトロ)駅\t徒歩12分\t\t日暮里・舎人ライナー\t熊野前(舎人ラ...,1K,34,NaN,26.32,5,バス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t洗面台独立,ガスコンロ／\tコンロ2口／\t給湯,インターネット対応／\t光ファイバー／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t24時間換気システム\...,駐輪場\t空有\tバイク置き場\t無,【コンビニ】 132m\t【スーパー】 452m\t【公園】 475m\t【スーパー】 55...,7,2年間,11.424094,5,1,0,0,1,0,0,"[12, 15, 12]",12,13.000000,荒川区,5557


In [21]:
train_v4['direction_countall'].value_counts()

17697    8909
9943     4995
8087     4026
6150     3098
6122     3080
5557     2749
3767     1966
2742     1326
2667     1321
Name: direction_countall, dtype: int64

In [13]:
v3 = pd.read_csv('../processed_data/train_v3.csv')
v3['direction']

0         南東
1        NaN
2          南
3          南
4          南
        ... 
31465      南
31466      西
31467      南
31468      南
31469     南東
Name: direction, Length: 31470, dtype: object